# <div align="center"><b> ETIQUETADO - PROYECTO FINAL </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomendado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al siguiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/ruta/de/archivo.ipynb">Ruta archivo</a></font></small></em> </div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [1]:
import os, sys, logging, datetime, re

sys.path.append(os.path.abspath("../"))  # Agregar el directorio padre al path

from dotenv import load_dotenv
load_dotenv("../.env.dev")

OPENCV_IO_MAX_IMAGE_PIXELS = 50000 * 50000  # Para imágenes grandes, ej: barrio3Ombues_20180801_dji_pc_3cm.jpg
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(OPENCV_IO_MAX_IMAGE_PIXELS)

import cv2 as cv
from apps_utils.logging import Logging

import apps_etiquetado.procesador_anotaciones_cvat as ProcesadorCVAT
import apps_etiquetado.procesador_anotaciones_mongodb as ProcesadorMongoDB

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [2]:
LOGGER = Logging().logger

LOGGER.info("Configuración cargada correctamente.")

2025-05-24 12:34:50,795 - root - INFO - <module> - Configuración cargada correctamente.


<!-- Colab -->
<!-- <div align="center"><img src="https://drive.google.com/uc?export=view&id=1QSNrTsz1hQbmZwpgwx0qpfpNtLW19Orm" width="600" alt="Figura 1: A data scientist is working on word generation using the Lord of the Rings lore. The image is dark and moody, with a focus on the scientist's computer screen. The screen displays a visualization the one ring, with a map of Middle Earth in the background. - Generada con DALL-E3"></div> -->

<!-- <div align="center"><img src="./ceia-materia/resources/portada.jpeg" width="600" alt="Figura 1: A data scientist playing with convolutional neural networks. - Generada con Microsoft Image Creator"></div>

<div align="center"><small><em>Figura 1: A data scientist playing with convolutional neural networks. - Generada con Microsoft Image Creator</em></small></div> -->

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Etiquetado                                                                                                                             |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | Herramientas y scripts para el etiquetado y guardado de los datos                                                                      |
| **Integrantes** | Bruno Masoller (brunomaso1@gmail.com)                                                                                                  |

</div>

## Consinga

El objetivo de este proyecto es brindar herramientas y scripts para el etiquetado y guardado de los datos.

## Resolución

Utilidades de Ultralytics:

- [https://docs.ultralytics.com/es/usage/simple-utilities/](https://docs.ultralytics.com/es/usage/simple-utilities/)

### Anotaciones

Para las anotaciones, entre varios formatos estudiados, se eligió el formato de COCO.

- [https://roboflow.com/formats/coco-json](https://roboflow.com/formats/coco-json)
- [https://docs.voxel51.com/recipes/convert_datasets.html](https://docs.voxel51.com/recipes/convert_datasets.html)
- [https://stackoverflow.com/questions/75927857/how-to-convert-coco-json-to-yolov8-segmentation-format](https://stackoverflow.com/questions/75927857/how-to-convert-coco-json-to-yolov8-segmentation-format)

### Interacción con google maps

#### Generación de mapa

Procesamiento de archivos KML:
- `fastkml` $\rightarrow$ [https://fastkml.readthedocs.io/en/latest/quickstart.html](https://fastkml.readthedocs.io/en/latest/quickstart.html)

Conversión de archivos KML a GeoJSON:
- `kml2geojson` $\rightarrow$ [https://pypi.org/project/kml2geojson/](https://pypi.org/project/kml2geojson/) | [https://github.com/mrcagney/kml2geojson](https://github.com/mrcagney/kml2geojson)

Visualización:
- `folium` $\rightarrow$ [https://python-visualization.github.io/folium/latest/](https://python-visualization.github.io/folium/latest/)
- `geojson.io` $\rightarrow$ [https://geojson.io/#map=2/0/20](https://geojson.io/#map=2/0/20)

Trabajo con archivos GeoJSON:
- `geojson` $\rightarrow$ [https://github.com/jazzband/geojson](https://github.com/jazzband/geojson) | [https://geojson.org/](https://geojson.org/)
- `geopandas` $\rightarrow$ [https://geopandas.org/](https://geopandas.org/)
- `shapely` $\rightarrow$ [https://shapely.readthedocs.io/en/latest/manual.html](https://shapely.readthedocs.io/en/latest/manual.html)

### Sincronizar anotaciones CVAT/MongoDB

In [3]:
# Probar conexiones a CVAT y MongoDB
try:
    ProcesadorCVAT.test_connection()
    LOGGER.info("Conexión a CVAT exitosa.")
except Exception as e:
    LOGGER.error(f"Error al conectar a CVAT: {e}")
    sys.exit(1)

try:
    ProcesadorMongoDB.test_connection()
    LOGGER.info("Conexión a MongoDB exitosa.")
except Exception as e:
    LOGGER.error(f"Error al conectar a MongoDB: {e}")
    sys.exit(1)

2025-05-24 12:34:52,295 - root - INFO - <module> - Conexión a CVAT exitosa.
2025-05-24 12:34:52,318 - root - INFO - test_connection - Conexión a la base de datos MongoDB exitosa.
2025-05-24 12:34:52,319 - root - INFO - <module> - Conexión a MongoDB exitosa.


In [4]:
# Desde CVAT
coco_annotations = ProcesadorCVAT.load_annotations_from_cvat(task_id=8)
ProcesadorMongoDB.save_coco_annotations(coco_annotations=coco_annotations, field_name="cvat")

2025-05-24 12:35:23,970 - cvat_sdk.core.client - INFO - prepare_file - Waiting for the server to prepare the file...
2025-05-24 12:35:29,106 - cvat_sdk.core.client - INFO - export_dataset - Dataset for task 8 has been downloaded to downloads\temp\cvat_task_8.zip
2025-05-24 12:35:29,143 - root - WARNING - save_coco_annotations - No se encontraron anotaciones para la imagen patches/AcostayLara_20240927_dji_rtk_5cm/AcostayLara_20240927_dji_rtk_5cm_patch_12.jpg.
2025-05-24 12:35:29,144 - root - WARNING - save_coco_annotations - No se encontraron anotaciones para la imagen patches/AcostayLara_20240927_dji_rtk_5cm/AcostayLara_20240927_dji_rtk_5cm_patch_14.jpg.
2025-05-24 12:35:29,144 - root - WARNING - save_coco_annotations - No se encontraron anotaciones para la imagen patches/AcostayLara_20240927_dji_rtk_5cm/AcostayLara_20240927_dji_rtk_5cm_patch_17.jpg.
2025-05-24 12:35:29,144 - root - WARNING - save_coco_annotations - No se encontraron anotaciones para la imagen patches/AcostayLara_20240

True

In [ ]:
# Desde un archivo
# coco_annotations = load_annotations_from_file(file_path="cvat.json")
# save_coco_annotations(coco_annotations=coco_annotations, field_name="cvat")

### Fixes

In [ ]:
def fix_image_width_height(image_id: str, width: int, height: int) -> bool:
    """Actualiza el ancho y alto de una imagen en la base de datos.

    Args:
        image_id (str): Identificador de la imagen a actualizar.
        width (int): Nuevo ancho de la imagen.
        height (int): Nuevo alto de la imagen.

    Returns:
        bool: True si la actualización fue exitosa, False en caso contrario.
    """
    try:
        imagenes = DB.get_collection("imagenes")
        result = imagenes.update_one(
            {"id": image_id},
            {
                "$set": {
                    "width": width,
                    "height": height,
                }
            },
        )
        if result.modified_count > 0:
            LOGGER.debug(f"Ancho y alto de la imagen {image_id} actualizados correctamente.")
            return True
        else:
            LOGGER.warning(f"No se encontraron cambios para la imagen {image_id}.")
            return False
    except Exception as e:
        LOGGER.error(f"Error al actualizar el ancho y alto de la imagen {image_id}: {e}")
        return False

In [ ]:
def fix_width_height_images():
    """Corrige el ancho y alto de las imágenes en la base de datos.

    Esta función descarga las imágenes desde MinIO, obtiene sus dimensiones (ancho y alto),
    actualiza estos valores en la base de datos y elimina las imágenes descargadas localmente.

    Pasos realizados:
    1. Configura el logger para registrar las actualizaciones.
    2. Obtiene todas las imágenes almacenadas en la base de datos.
    3. Descarga cada imagen desde MinIO si está marcada como descargada.
    4. Calcula las dimensiones de la imagen descargada.
    5. Actualiza las dimensiones en la base de datos.
    6. Elimina la imagen descargada localmente.

    Raises:
        OSError: Si ocurre un error al eliminar la imagen descargada localmente.
    """

    set_log_to_file(f"updates_{datetime.date.today()}.log")
    LOGGER.setLevel(logging.INFO)
    LOGGER.info("Iniciando actualización de imágenes y parches.")
    # Actualizar el width y height de las imágenes en la base de datos
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")

    # 2 - Para cada imagen:
    for image in imagenes.find():
        image_id = image["id"]
        downloaded: bool = image["downloaded"]
        if not downloaded:
            LOGGER.warning(f"La imagen {image_id} no está descargada, se omitirá.")
            continue
        LOGGER.info(f"Actualizando imagen {image_id}...")
        # 2.1 - Descargar la imagen desde MinIO

        jpg_path = download_image_from_minio(image_id)
        LOGGER.info(f"Imagen {image_id} descargada correctamente.")
        # 2.2 - Obtener el ancho y alto de la imagen descargada
        img = cv.imread(jpg_path)
        height, width = img.shape[:2]
        # 2.3 - Actualizar el ancho y alto de la imagen en la base de datos
        fix_image_width_height(image_id, width, height)
        LOGGER.info(f"Ancho y alto de la imagen {image_id} actualizados correctamente.")
        # 2.4 - Eliminar la imagen descargada
        try:
            os.remove(jpg_path)
            LOGGER.info(f"Imagen {jpg_path} eliminada correctamente.")
        except OSError as e:
            LOGGER.warning(f"Error al eliminar la imagen {jpg_path}: {e}")
            continue
    LOGGER.info("Actualización de imágenes y parches finalizada.")

In [ ]:
def fix_dates_images():
    set_log_to_file(f"fix_dates_images_{datetime.date.today()}.log")
    LOGGER.setLevel(logging.INFO)
    LOGGER.info("Iniciando actualización de fechas de imágenes.")
    # Actualizar el width y height de las imágenes en la base de datos
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")
    date_format = "%d/%m/%Y"

    # 2 - Para cada imagen:
    for image in imagenes.find():
        # 2.1 - Obtener el título de la imagen
        image_title = image["title"]

        # 2.2 - Extraer la fecha del título de la imagen
        match = re.search(r"\d{2}/\d{2}/\d{4}", image_title)
        date_str = match.group(0) if match else None
        if not date_str:
            LOGGER.warning(f"No se encontró una fecha válida en el título de la imagen {image_title}.")
            continue

        # 2.3 - Actualizar la fecha de captura en la base de datos
        try:
            date_captured = datetime.datetime.strptime(date_str, date_format)
            imagenes.update_one(
                {"id": image["id"]},
                {
                    "$set": {
                        "date_captured": date_captured,
                    }
                },
            )
            LOGGER.info(f"Fecha de captura de la imagen {image_title} actualizada correctamente.")
        except Exception as e:
            LOGGER.warning(f"Error al actualizar la fecha de captura: {e}")

    LOGGER.info("Actualización de fechas de imágenes finalizada.")

In [ ]:
def fix_patch_name():
    """Corrige el nombre de los parches en la base de datos.
    Le saca el .jpg al final del nombre del parche y lo actualiza en la base de datos.
    """
    set_log_to_file(f"fix_patch_name_{datetime.date.today()}.log")
    LOGGER.setLevel(logging.INFO)
    LOGGER.info("Iniciando actualización de nombres de parches.")
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")

    # 2 - Para cada imagen:
    for image in imagenes.find():
        image_id = image["id"]
        # 2.1 - Obtener los parches de la imagen
        try:
            patches = image["patches"]
        except KeyError:
            LOGGER.warning(f"No se encontraron parches para la imagen {image_id}.")
            continue
        # 2.2 - Para cada parche:
        for patch in patches:
            patch_name = patch["patch_name"]
            if patch_name.endswith(".jpg"):
                new_patch_name = patch_name[:-4]
                # 2.3 - Actualizar el nombre del parche en la base de datos
                imagenes.update_one(
                    {"id": image_id, "patches.patch_name": patch_name},
                    {"$set": {"patches.$.patch_name": new_patch_name}},
                )
                LOGGER.info(f"Nombre del parche {patch_name} actualizado a {new_patch_name}.")
            else:
                LOGGER.warning(f"El parche {patch_name} no tiene .jpg al final.")
    LOGGER.info("Actualización de nombres de parches finalizada.")